In [1]:
%load_ext autoreload
%autoreload 2

from theano.sandbox import cuda
cuda.use('gpu0')

from luvgg16 import VGG16
import numpy as np
import os 
import pandas as pd

np.set_printoptions(precision=4, linewidth=100)

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/home/lukaka/anaconda2/envs/py3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
NB_HOME = os.path.dirname(os.getcwd())
DATA_HOME = NB_HOME + '/data/redux'

MODEL_WEIGHT_PATH = DATA_HOME+'/result/vgg_model.h5'
SUBMIT_FILE_PATH = DATA_HOME+'/result/kaggle_submit.csv'

TEST_PATH = DATA_HOME + '/test/'

print(TEST_PATH)

/home/lukaka/Desktop/fastai-DL-course/deeplearning1/nbs/data/redux/test/


In [3]:
vgg = VGG16(pretrained_weight_path=MODEL_WEIGHT_PATH)

model loaded from path=/home/lukaka/Desktop/fastai-DL-course/deeplearning1/nbs/data/redux/result/vgg_model.h5


In [4]:
test_batch = vgg.get_batch_generator(TEST_PATH, shuffle=False)
test_filenames = test_batch.filenames

Found 12500 images belonging to 1 classes.


In [5]:
test_filenames[:5]

['unknown/64.jpg',
 'unknown/6693.jpg',
 'unknown/8431.jpg',
 'unknown/7999.jpg',
 'unknown/762.jpg']

In [ ]:
test_pred = vgg.predict_prob_generator(TEST_PATH)

Found 12500 images belonging to 1 classes.


In [54]:
clipped_pred = np.clip(test_pred, 0.01, 0.99)

In [55]:
ids = np.array([int(f[8:f.find('.')]) for f in test_filenames])

In [56]:
submit = pd.DataFrame({'id': ids, 'label': clipped_pred})

In [57]:
submit.describe()

,id,label
count,12500.000000,12500.000000
mean,6250.500000,0.500794
std,3608.583517,0.485933
min,1.000000,0.010000
25%,3125.750000,0.010000
50%,6250.500000,0.576305
75%,9375.250000,0.990000
max,12500.000000,0.990000


In [58]:
submit.to_csv(SUBMIT_FILE_PATH, index=False, float_format='%.5f')